<a href="https://colab.research.google.com/github/benjaminbrown038/Amazon/blob/main/notebooks/amazon/amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon

## Computer Vision

- Image Classification
- Object Detection
- Semantic Segmentation
- Instance Segmentation
- Image Embedding

## Image Classification

In [ ]:
!pip3 install sagemaker ipywidgets --upgrade --quiet
import sagemaker, boto3, json, IPython

from sagemaker import get_execution_role, image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from sagemaker.estimator import Estimator
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter

from IPython.core.display import HTML
import ipywidgets as widgets

In [ ]:
aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

In [ ]:
(model_id,model_version) =("pytorch-ic-mobilenet-v2","*")

In [ ]:
model_id

In [ ]:
model_version

In [ ]:
boto3.client("s3").download_file(f"jumpstart-cache-prod-{aws_region}", "models_manifest.json", "models_manifest.json")

In [ ]:
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)
model_list

In [ ]:
ic_models_all_versions, ic_models = [model["model_id"] for model in model_list if "-ic-" in model["model_id"]], [][ic_models.append(model) for model in ic_models_all_versions if model not in ic_models]

In [ ]:
ic_models_all_versions

In [ ]:
ic_models

In [ ]:
dropdown = widgets.Dropdown(
    options=ic_models,
    value=model_id,
    description="JumpStart Image Classification Models:",
    style={"description_width": "initial"},
    layout={"width": "max-content"})
dropdown

In [ ]:
display(IPython.display.Markdown("## Select a JumpStart pre-trained model from the dropdown below"))

In [ ]:
display(dropdown)

In [ ]:
infer_model_id, infer_model_version = dropdown.value, "*"

In [ ]:
infer_model_id

In [ ]:
infer_model_version

In [ ]:
endpoint_name = name_from_base(f"jumpstart-example-{infer_model_id}")
endpoint_name

In [ ]:
inference_instance_type = "ml.m5.xlarge"
inference_instance_type

In [ ]:
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=infer_model_id,
    model_version=infer_model_version,
    instance_type=inference_instance_type)
deploy_image_uri

In [ ]:
deploy_source_uri = script_uris.retrieve(model_id=infer_model_id, model_version=infer_model_version, script_scope="inference")
deploy_source_uri

In [ ]:
base_model_uri = model_uris.retrieve(model_id=infer_model_id, model_version=infer_model_version, model_scope="inference")

'''
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=base_model_uri,
    entry_point="inference.py",
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name)
'''
base_model_uri

In [ ]:
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name)
base_model_predictor

In [ ]:
s3_bucket = f"jumpstart-cache-prod-{aws_region}"
s3_bucket

In [ ]:
key_prefix = "inference-notebook-assets"
key_prefix

In [ ]:
def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)

In [ ]:
images = {"img1.jpg": "cat.jpg", "img2.jpg": "dog.jpg"}
images

In [ ]:
download_from_s3(images)

In [ ]:
def predict_top_k_labels(probabilities, labels, k):
    topk_prediction_ids = sorted(range(len(probabilities)), key=lambda index: probabilities[index], reverse=True)[:k]
    topk_class_labels = ", ".join([labels[id] for id in topk_prediction_ids])
    return topk_class_labels

In [ ]:
for image_filename in images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = base_model_predictor.predict(img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"})
    model_predictions = json.loads(query_response)
    labels, probabilities = model_predictions["labels"], model_predictions["probabilities"]
    top5_class_labels = predict_top_k_labels(probabilities, labels, 5)
    display(HTML(f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
                 f"<figcaption>Top-5 predictions: {top5_class_labels} </figcaption>"))

In [ ]:
base_model_predictor.delete_model()

In [ ]:
base_model_predictor.delete_endpoint()

In [ ]:
model_id, model_version = dropdown.value, "*"

In [ ]:
model_id

In [ ]:
model_version

In [ ]:
training_instance_type = "ml.p3.2xlarge"
training_instance_type

In [ ]:
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type)
train_image_uri

In [ ]:
train_source_uri = script_uris.retrieve(model_id=model_id, model_version=model_version, script_scope="training")
train_source_uri

In [ ]:
train_model_uri = model_uris.retrieve(model_id=model_id, model_version=model_version, model_scope="training")
train_model_uri

In [ ]:
training_data_bucket = f"jumpstart-cache-prod-{aws_region}"
training_data_bucket

In [ ]:
training_data_prefix = "training-datasets/tf_flowers/"
training_data_prefix

In [ ]:
training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"
training_dataset_s3_path

In [ ]:
output_bucket = sess.default_bucket()
output_bucket

In [ ]:
output_prefix = "jumpstart-example-ic-training"
output_prefix

In [ ]:
s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"
s3_output_location

In [ ]:
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)
hyperparameters

In [ ]:
hyperparameters["epochs"] = "5"

In [ ]:
print(hyperparameters)

In [ ]:
use_amt = True
use_amt

In [ ]:
metric_definitions_per_model = {
    "tensorflow": {"metrics": [{"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"}],
                   "type": "Maximize"},
    "pytorch": {"metrics": [{"Name": "val_accuracy", "Regex": "val Acc: ([0-9\\.]+)"}],
                "type": "Maximize"}}
metric_definitions_per_model

In [ ]:
hyperparameter_ranges = {"adam-learning-rate": ContinuousParameter(0.0001, 0.1, scaling_type="Logarithmic")}
hyperparameter_ranges

In [ ]:
max_jobs = 6
max_jobs

In [ ]:
max_parallel_jobs = 2
max_parallel_jobs

In [ ]:
training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")
training_job_name

In [ ]:
ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name)
ic_estimator

In [ ]:
if use_amt:
    metric_definitions = next(value for key, value in metric_definitions_per_model.items() if model_id.startswith(key))
    hp_tuner = HyperparameterTuner(
        ic_estimator,
        metric_definitions["metrics"][0]["Name"],
        hyperparameter_ranges,
        metric_definitions["metrics"],
        max_jobs=max_jobs,
        max_parallel_jobs=max_parallel_jobs,
        objective_type=metric_definitions["type"],
        base_tuning_job_name=training_job_name)
    hp_tuner.fit({"training": training_dataset_s3_path})
else:
    ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)

In [ ]:
inference_instance_type = "ml.m5.xlarge"
inference_instance_type

In [ ]:
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type)
deploy_image_uri

In [ ]:
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference")
deploy_source_uri

In [ ]:
endpoint_name = name_from_base(f"jumpstart-example-FT-{model_id}-")
endpoint_name

In [ ]:
finetuned_predictor = (hp_tuner if use_amt else ic_estimator).deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name)
finetuned_predictor

In [ ]:
s3_bucket = f"jumpstart-cache-prod-{aws_region}"
s3_bucket

In [ ]:
key_prefix = "training-datasets/tf_flowers"
key_prefix

In [ ]:
def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)

In [ ]:
flower_images = {"img1.jpg": "roses/10503217854_e66a804309.jpg",
                 "img2.jpg": "sunflowers/1008566138_6927679c8a.jpg"}
flower_images

In [ ]:
download_from_s3(flower_images)

In [ ]:
for image_filename in flower_images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = finetuned_predictor.predict(img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"})
    model_predictions = json.loads(query_response)
    predicted_label = model_predictions["predicted_label"]
    display(HTML(f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
                 f"<figcaption>Predicted Label: {predicted_label}</figcaption>"))

In [ ]:
finetuned_predictor.delete_model()

In [ ]:
finetuned_predictor.delete_endpoint()

In [ ]:
if use_amt:
    sage_client = boto3.Session().client("sagemaker")
    tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=hp_tuner._current_job_name)
    last_training_job_name = tuning_job_result["BestTrainingJob"]["TrainingJobName"]
else:
    last_training_job_name = ic_estimator._current_job_name

In [ ]:
last_trained_model_path = f"{s3_output_location}/{last_training_job_name}/output/model.tar.gz"
last_trained_model_path

In [ ]:
incremental_train_output_prefix = "jumpstart-example-ic-incremental-training"
incremental_train_output_prefix

In [ ]:
incremental_s3_output_location = f"s3://{output_bucket}/{incremental_train_output_prefix}/output"
incremental_s3_output_location

In [ ]:
incremental_training_job_name = name_from_base(f"jumpstart-example-{model_id}-incremental-training")
incremental_training_job_name

In [ ]:
incremental_train_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=last_trained_model_path,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=incremental_s3_output_location,
    base_job_name=incremental_training_job_name)
incremental_train_estimator

In [ ]:
incremental_train_estimator.fit({"training": training_dataset_s3_path}, logs=True)

## Object Detection

In [ ]:
!pip3 install sagemaker ipywidgets --upgrade --quiet

import sagemaker, boto3, json
import IPython
from IPython.core.display import HTML
import ipywidgets as widgets

from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from sagemaker import get_execution_role, image_uris, model_uris, script_uris, hyperparameters
from sagemaker import hyperparameters
from sagemaker.tuner import ContinuousParameter, HyperparameterTuner
from sagemaker.estimator import Estimator

In [ ]:
aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

In [ ]:
(model_id,model_version,) = ("pytorch-ic-mobilenet-v2","*")

In [ ]:
boto3.client("s3").download_file(f"jumpstart-cache-prod-{aws_region}", "models_manifest.json", "models_manifest.json")

In [ ]:
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)
model_list

In [ ]:
ic_models_all_versions, ic_models = [model["model_id"] for model in model_list if "-ic-" in model["model_id"]], [][ic_models.append(model) for model in ic_models_all_versions if model not in ic_models]

In [ ]:
ic_models_all_versions

In [ ]:
ic_models

In [ ]:
dropdown = widgets.Dropdown(
    options=ic_models,
    value=model_id,
    description="JumpStart Image Classification Models:",
    style={"description_width": "initial"},
    layout={"width": "max-content"})
dropdown

In [ ]:
display(IPython.display.Markdown("## Select a JumpStart pre-trained model from the dropdown below"))

In [ ]:
display(dropdown)

In [ ]:
infer_model_id, infer_model_version = dropdown.value, "*"

In [ ]:
infer_model_id

In [ ]:
infer_model_version

In [ ]:
endpoint_name = name_from_base(f"jumpstart-example-{infer_model_id}")
endpoint_name

In [ ]:
inference_instance_type = "ml.m5.xlarge"
inference_instance_type

In [ ]:
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=infer_model_id,
    model_version=infer_model_version,
    instance_type=inference_instance_type)
deploy_image_uri

In [ ]:
deploy_source_uri = script_uris.retrieve(model_id=infer_model_id, model_version=infer_model_version, script_scope="inference")
deploy_source_uri

In [ ]:
base_model_uri = model_uris.retrieve(model_id=infer_model_id, model_version=infer_model_version, model_scope="inference")
base_model_uri

In [ ]:
model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=base_model_uri,
    entry_point="inference.py",
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name)
model

In [ ]:
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name)
base_model_predictor

In [ ]:
s3_bucket = f"jumpstart-cache-prod-{aws_region}"
s3_bucket

In [ ]:
key_prefix = "inference-notebook-assets"
key_prefix

In [ ]:
def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)

In [ ]:
images = {"img1.jpg": "cat.jpg", "img2.jpg": "dog.jpg"}
images

In [ ]:
download_from_s3(images)

In [ ]:
def predict_top_k_labels(probabilities, labels, k):
    topk_prediction_ids = sorted(range(len(probabilities)), key=lambda index: probabilities[index], reverse=True)[:k]
    topk_class_labels = ", ".join([labels[id] for id in topk_prediction_ids])
    return topk_class_labels

In [ ]:
for image_filename in images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = base_model_predictor.predict(img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"})
    model_predictions = json.loads(query_response)
    labels, probabilities = model_predictions["labels"], model_predictions["probabilities"]
    top5_class_labels = predict_top_k_labels(probabilities, labels, 5)
    display(HTML(f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
                 f"<figcaption>Top-5 predictions: {top5_class_labels} </figcaption>"))

In [ ]:
base_model_predictor.delete_model()

In [ ]:
base_model_predictor.delete_endpoint()

In [ ]:
model_id, model_version = dropdown.value, "*"

In [ ]:
model_id

In [ ]:
model_version

In [ ]:
training_instance_type = "ml.p3.2xlarge"
training_instance_type

In [ ]:
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type)
train_image_uri

In [ ]:
train_source_uri = script_uris.retrieve(model_id=model_id, model_version=model_version, script_scope="training")
train_source_uri

In [ ]:
train_model_uri = model_uris.retrieve(model_id=model_id, model_version=model_version, model_scope="training")
train_model_uri

In [ ]:
training_data_bucket = f"jumpstart-cache-prod-{aws_region}"
training_data_bucket

In [ ]:
training_data_prefix = "training-datasets/tf_flowers/"
training_data_prefix

In [ ]:
training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"
training_dataset_s3_path

In [ ]:
output_bucket = sess.default_bucket()
output_bucket

In [ ]:
output_prefix = "jumpstart-example-ic-training"
output_prefix

In [ ]:
s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"
s3_output_location

In [ ]:
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)
hyperparameters

In [ ]:
hyperparameters["epochs"] = "5"

In [ ]:
print(hyperparameters)

In [ ]:
use_amt = True
use_amt

In [ ]:
metric_definitions_per_model = {
    "tensorflow": {
        "metrics": [{"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"}],
        "type": "Maximize"},
    "pytorch": {
        "metrics": [{"Name": "val_accuracy", "Regex": "val Acc: ([0-9\\.]+)"}],
        "type": "Maximize"}}
metric_definitions_per_model

In [ ]:
hyperparameter_ranges = {"adam-learning-rate": ContinuousParameter(0.0001, 0.1, scaling_type="Logarithmic")}
hyperparameter_ranges

In [ ]:
max_jobs = 6
max_jobs

In [ ]:
max_parallel_jobs = 2
max_parallel_jobs

In [ ]:
training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")
training_job_name

In [ ]:
ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name)
ic_estimator

In [ ]:
if use_amt:
    metric_definitions = next(
        value for key, value in metric_definitions_per_model.items() if model_id.startswith(key))
    hp_tuner = HyperparameterTuner(
        ic_estimator,
        metric_definitions["metrics"][0]["Name"],
        hyperparameter_ranges,
        metric_definitions["metrics"],
        max_jobs=max_jobs,
        max_parallel_jobs=max_parallel_jobs,
        objective_type=metric_definitions["type"],
        base_tuning_job_name=training_job_name)
    hp_tuner.fit({"training": training_dataset_s3_path})
else:
    ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)

In [ ]:
inference_instance_type = "ml.m5.xlarge"
inference_instance_type

In [ ]:
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type)
deploy_image_uri

In [ ]:
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference")
deploy_source_uri

In [ ]:
endpoint_name = name_from_base(f"jumpstart-example-FT-{model_id}-")
endpoint_name

In [ ]:
finetuned_predictor = (hp_tuner if use_amt else ic_estimator).deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name)
finetuned_predictor

In [ ]:
s3_bucket = f"jumpstart-cache-prod-{aws_region}"
s3_bucket

In [ ]:
key_prefix = "training-datasets/tf_flowers"
key_prefix

In [ ]:
def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)

In [ ]:
flower_images = {"img1.jpg": "roses/10503217854_e66a804309.jpg",
                 "img2.jpg": "sunflowers/1008566138_6927679c8a.jpg"}
flower_images

In [ ]:
download_from_s3(flower_images)

In [ ]:
for image_filename in flower_images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = finetuned_predictor.predict(img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"})
    model_predictions = json.loads(query_response)
    predicted_label = model_predictions["predicted_label"]
    display(HTML(f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
                 f"<figcaption>Predicted Label: {predicted_label}</figcaption>"))

In [ ]:
finetuned_predictor.delete_model()

In [ ]:
finetuned_predictor.delete_endpoint()

In [ ]:
if use_amt:
    sage_client = boto3.Session().client("sagemaker")
    tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=hp_tuner._current_job_name)
    last_training_job_name = tuning_job_result["BestTrainingJob"]["TrainingJobName"]
else:
    last_training_job_name = ic_estimator._current_job_name

In [ ]:
last_trained_model_path = f"{s3_output_location}/{last_training_job_name}/output/model.tar.gz"
last_trained_model_path

In [ ]:
incremental_train_output_prefix = "jumpstart-example-ic-incremental-training"
incremental_train_output_prefix

In [ ]:
incremental_s3_output_location = f"s3://{output_bucket}/{incremental_train_output_prefix}/output"
incremental_s3_output_location

In [ ]:
incremental_training_job_name = name_from_base(f"jumpstart-example-{model_id}-incremental-training")
incremental_training_job_name

In [ ]:
incremental_train_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=last_trained_model_path,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=incremental_s3_output_location,
    base_job_name=incremental_training_job_name)
incremental_train_estimator

In [ ]:
incremental_train_estimator.fit({"training": training_dataset_s3_path}, logs=True)

# Semantic Segmentation

In [ ]:
!pip3 install sagemaker ipywidgets --upgrade --quiet
from ipywidgets import Dropdown
import json
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image


import sagemaker, boto3, json
from sagemaker import get_execution_role
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

from sagemaker import image_uris, model_uris, script_uris
from sagemaker import hyperparameters
from sagemaker.tuner import ContinuousParameter
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

In [ ]:
aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

In [ ]:
boto3.client("s3").download_file(f"jumpstart-cache-prod-{aws_region}", "models_manifest.json", "models_manifest.json")

In [ ]:
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)
model_list

In [ ]:
semseg_models = []
semseg_models

In [ ]:
for model in model_list:
    model_id = model["model_id"]
    if "-semseg-" in model_id and model_id not in semseg_models:
        semseg_models.append(model_id)

In [ ]:
print(f"\033[38;5;2mChose a model: \033[0;0m\n")

In [ ]:
model_dropdown = Dropdown(
    options=semseg_models,
    value="mxnet-semseg-fcn-resnet50-ade",
    description="",
    style={"description_width": "initial"},
    layout={"width": "max-content"})
model_dropdown

In [ ]:
display(model_dropdown)with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

In [ ]:
model_id, model_version = model_dropdown.value, "*"

In [ ]:
model_id

In [ ]:
model_version

In [ ]:
endpoint_name = name_from_base(f"jumpstart-example-infer-{model_id}")
endpoint_name

In [ ]:
inference_instance_type = "ml.p2.xlarge"
inference_instance_type

In [ ]:
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type)
deploy_image_uri

In [ ]:
deploy_source_uri = script_uris.retrieve(model_id=model_id, model_version=model_version, script_scope="inference")
deploy_source_uri

In [ ]:
base_model_uri = model_uris.retrieve(model_id=model_id, model_version=model_version, model_scope="inference")
base_model_uri

In [ ]:
model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=base_model_uri,
    entry_point="inference.py",
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name)
model

In [ ]:
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    predictor_cls=Predictor,
    endpoint_name=endpoint_name)
base_model_predictor

In [ ]:
jumpstart_assets_s3_bucket = f"jumpstart-cache-prod-{aws_region}"
jumpstart_assets_s3_bucket

In [ ]:
pedestrian_img_key_prefix = "inference-notebook-assets"
pedestrian_img_key_prefix

In [ ]:
pedestrian_img = "img_pedestrian.png"
pedestrian_img

In [ ]:
boto3.client("s3").download_file(jumpstart_assets_s3_bucket, f"{pedestrian_img_key_prefix}/{pedestrian_img}", pedestrian_img)

In [ ]:
def query(model_predictor, image_file_name):
    """Query the model predictor."""

    with open(image_file_name, "rb") as file:
        input_img_rb = file.read()

    query_response = model_predictor.predict(
        input_img_rb,
        {
            "ContentType": "application/x-image",
            "Accept": "application/json;verbose",
        },
    )
    return query_response

In [ ]:
def parse_response(query_response):
    """Parse response and return predictions as well as the set of all labels and object labels present in the image."""
    response_dict = json.loads(query_response)
    return response_dict["predictions"], response_dict["labels"], response_dict["image_labels"]

In [ ]:
query_response = query(base_model_predictor, pedestrian_img)
query_response

In [ ]:
predictions, labels, image_labels = parse_response(query_response)


In [ ]:
predictions

In [ ]:
labels

In [ ]:
image_labels

In [ ]:
print("Objects present in the picture:", image_labels)

In [ ]:
def getvocpalette(num_cls):
    """Get a color palette."""

    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab > 0:
            palette[j * 3 + 0] |= ((lab >> 0) & 1) << (7 - i)
            palette[j * 3 + 1] |= ((lab >> 1) & 1) << (7 - i)
            palette[j * 3 + 2] |= ((lab >> 2) & 1) << (7 - i)
            i = i + 1
            lab >>= 3
    return palette

In [ ]:
def display_predictions(predictions):
    palette = getvocpalette(256)
    npimg = np.array(predictions)
    npimg[npimg == -1] = 255
    mask = Image.fromarray(npimg.astype("uint8"))
    mask.putpalette(palette)
    plt.imshow(mask)

In [ ]:
display_predictions(predictions)

In [ ]:
base_model_predictor.delete_model()

In [ ]:
base_model_predictor.delete_endpoint()

In [ ]:
train_scope = "training"
train_scope

In [ ]:
training_instance_type = "ml.p3.2xlarge"
training_instance_type

In [ ]:
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope=train_scope,
    instance_type=training_instance_type)
train_image_uri

In [ ]:
train_source_uri = script_uris.retrieve(model_id=model_id, model_version=model_version, script_scope=train_scope)
train_source_uri

In [ ]:
train_model_uri = model_uris.retrieve(model_id=model_id, model_version=model_version, model_scope=train_scope)
train_model_uri

In [ ]:
training_data_bucket = f"jumpstart-cache-prod-{aws_region}"
training_data_bucket

In [ ]:
training_data_prefix = "training-datasets/PennFudanPed_SemSeg/"
training_data_prefix

In [ ]:
training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"
training_dataset_s3_path

In [ ]:
output_bucket = sess.default_bucket()
output_bucket

In [ ]:
output_prefix = "jumpstart-example-semseg-training"
output_prefix

In [ ]:
s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"
s3_output_location

In [ ]:
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)
hyperparameters

In [ ]:
hyperparameters["epochs"] = "4"

In [ ]:
print(hyperparameters)

In [ ]:
use_amt = True
use_amt

In [ ]:
metric_definitions_per_model = {
    "mxnet": {"metrics": [{"Name": "val_loss", "Regex": "validation loss=([0-9\\.]+)"}],
              "type": "Minimize"}}
metric_definitions_per_model

In [ ]:
hyperparameter_ranges = {"adam-learning-rate": ContinuousParameter(0.0001, 0.1, scaling_type="Logarithmic")}
hyperparameter_ranges

In [ ]:
max_jobs = 6
max_jobs

In [ ]:
max_parallel_jobs = 2
max_parallel_jobs

In [ ]:
training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")
training_job_name

In [ ]:
semseg_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",  # Entry-point file in source_dir and present in train_source_uri.
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name)
semseg_estimator

In [ ]:
if use_amt:
    metric_definitions = next(
        value for key, value in metric_definitions_per_model.items() if model_id.startswith(key))
    hp_tuner = HyperparameterTuner(
        semseg_estimator,
        metric_definitions["metrics"][0]["Name"],
        hyperparameter_ranges,
        metric_definitions["metrics"],
        max_jobs=max_jobs,
        max_parallel_jobs=max_parallel_jobs,
        objective_type=metric_definitions["type"],
        base_tuning_job_name=training_job_name)
    hp_tuner.fit({"training": training_dataset_s3_path})
else:
    semseg_estimator.fit({"training": training_dataset_s3_path}, logs=True)

In [ ]:
inference_instance_type = "ml.p2.xlarge"
inference_instance_type

In [ ]:
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type)
deploy_image_uri

In [ ]:
deploy_source_uri = script_uris.retrieve(model_id=model_id, model_version=model_version, script_scope="inference")
deploy_source_uri

In [ ]:
endpoint_name = name_from_base(f"jumpstart-example-FT-{model_id}-")
endpoint_name

In [ ]:
finetuned_predictor = (hp_tuner if use_amt else semseg_estimator).deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name)
finetuned_predictor

In [ ]:
jumpstart_assets_bucket = f"jumpstart-cache-prod-{aws_region}"
jumpstart_assets_bucket

In [ ]:
pedestrian_image_key = "training-datasets/PennFudanPed_SemSeg/images"
pedestrian_image_key

In [ ]:
pedestrian_image_file_name = "FudanPed00001.png"
pedestrian_image_file_name

In [ ]:
boto3.client("s3").download_file(jumpstart_assets_bucket,f"{pedestrian_image_key}/{pedestrian_image_file_name}",pedestrian_image_file_name)

In [ ]:
query_response = query(finetuned_predictor, pedestrian_img)
query_response

In [ ]:
predictions, labels, image_labels = parse_response(query_response)

In [ ]:
predictions

In [ ]:
labels

In [ ]:
image_labels

In [ ]:
display_predictions(predictions)

In [ ]:
finetuned_predictor.delete_model()

In [ ]:
finetuned_predictor.delete_endpoint()

# Image Embedding

In [ ]:
!pip3 install sagemaker ipywidgets --upgrade --quiet

import sagemaker, boto3, json
from ipywidgets import Dropdown
from IPython.core.display import HTML

from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

from sagemaker import get_execution_role

In [ ]:
aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

In [ ]:
boto3.client("s3").download_file(f"jumpstart-cache-prod-{aws_region}", "models_manifest.json", "models_manifest.json")

In [ ]:
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)
model_list

In [ ]:
image_embedding_models = []
image_embedding_models

In [ ]:
for model in model_list:
    model_id = model["model_id"]
    if "-icembedding-" in model_id and model_id not in image_embedding_models:
        image_embedding_models.append(model_id)

In [ ]:
model_dropdown = Dropdown(
    options=image_embedding_models,
    value="tensorflow-icembedding-bit-m-r101x1-ilsvrc2012-featurevector-1",
    description="Select a model",
    style={"description_width": "initial"},
    layout={"width": "max-content"})
model_dropdown

In [ ]:
display(model_dropdown)

In [ ]:
model_id, model_version = model_dropdown.value, "*"

In [ ]:
model_id

In [ ]:
model_version

In [ ]:
endpoint_name = name_from_base(f"jumpstart-example-infer-{model_id}")
endpoint_name

In [ ]:
inference_instance_type = "ml.p2.xlarge"
inference_instance_type

In [ ]:
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type)
deploy_image_uri

In [ ]:
deploy_source_uri = script_uris.retrieve(model_id=model_id, model_version=model_version, script_scope="inference")
deploy_source_uri

In [ ]:
model_uri = model_uris.retrieve(model_id=model_id, model_version=model_version, model_scope="inference")
model_uri

In [ ]:
model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=model_uri,
    entry_point="inference.py",
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name)
model

In [ ]:
model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    predictor_cls=Predictor,
    endpoint_name=endpoint_name)
model_predictor

In [ ]:
region = boto3.Session().region_name
region

In [ ]:
s3_bucket = f"jumpstart-cache-prod-{region}"
s3_bucket

In [ ]:
key_prefix = "inference-notebook-assets"
key_prefix

In [ ]:
s3 = boto3.client("s3")
s3

In [ ]:
def download_from_s3(key_filenames):
    for key_filename in key_filenames:
        s3.download_file(s3_bucket, f"{key_prefix}/{key_filename}", key_filename)

In [ ]:
cat_jpg, dog_jpg = "cat.jpg", "dog.jpg"

In [ ]:
cat_jpg

In [ ]:
dog_jpg

In [ ]:
download_from_s3(key_filenames=[cat_jpg, dog_jpg])

In [ ]:
def download_from_s3(key_filenames):
    for key_filename in key_filenames:
        s3.download_file(s3_bucket, f"{key_prefix}/{key_filename}", key_filename)

In [ ]:
download_from_s3(key_filenames=[cat_jpg, dog_jpg])

In [ ]:
def query(model_predictor, image_file_name):
    with open(image_file_name, "rb") as file:
        input_img_rb = file.read()
    query_response = model_predictor.predict(input_img_rb,
        {   "ContentType": "application/x-image",
            "Accept": "application/json",
        },)
    return query_response

In [ ]:
def parse_response(query_response):
    model_predictions = json.loads(query_response)
    translation_text = model_predictions["embedding"]
    return translation_text

In [ ]:
for img_name in [cat_jpg, dog_jpg]:
    query_response = query(model_predictor, img_name)
    embedding = parse_response(query_response)
    first_5element_embeddings = "{" + ", ".join([str(id) for id in embedding[:5]]) + "}"
    display(HTML(f'<img src={img_name} alt={img_name} align="left" style="width: 250px;"/>'
                 f"<figcaption>First-5 elements of the feature vector (embedding) are: {first_5element_embeddings}</figcaption>"
                 f"<figcaption>Total length of the feature vector (embedding) is: {len(embedding)}</figcaption>"))

In [ ]:
model_predictor.delete_model()

In [ ]:
model_predictor.delete_endpoint()

# Instant Segmentation

In [ ]:
!pip install sagemaker ipywidgets --upgrade --quiet
import sagemaker, boto3, json
from sagemaker import get_execution_role
from ipywidgets import Dropdown

from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

import json
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

In [ ]:
boto3.client("s3").download_file(f"jumpstart-cache-prod-{aws_region}", "models_manifest.json", "models_manifest.json")

In [ ]:
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

In [ ]:
is_models = []
is_models

In [ ]:
for model in model_list:
    model_id = model["model_id"]
    if "-is-" in model_id and model_id not in is_models:
        is_models.append(model_id)

In [ ]:
print(f"\033[38;5;2mChose a model for inference: \033[0;0m\n")

In [ ]:
model_dropdown = Dropdown(
    options=is_models,
    value="mxnet-is-mask-rcnn-fpn-resnet101-v1d-coco",
    description="",
    style={"description_width": "initial"},
    layout={"width": "max-content"})
model_dropdown

In [ ]:
display(model_dropdown)

In [ ]:
model_id, model_version = model_dropdown.value, "*"

In [ ]:
model_id

In [ ]:
model_version

In [ ]:
endpoint_name = name_from_base(f"jumpstart-example-infer-{model_id}")
endpoint_name

In [ ]:
inference_instance_type = "ml.p2.xlarge"
inference_instance_type

In [ ]:
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type)
deploy_image_uri

In [ ]:
deploy_source_uri = script_uris.retrieve(model_id=model_id, model_version=model_version, script_scope="inference")
deploy_source_uri

In [ ]:
base_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference")
base_model_uri

In [ ]:
model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=base_model_uri,
    entry_point="inference.py",
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name)
model

In [ ]:
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    predictor_cls=Predictor,
    endpoint_name=endpoint_name)
base_model_predictor

In [ ]:
jumpstart_assets_s3_bucket = f"jumpstart-cache-prod-{aws_region}"
jumpstart_assets_s3_bucket

In [ ]:
pedestrian_img_key_prefix = "inference-notebook-assets"
pedestrian_img_key_prefix

In [ ]:
pedestrian_img = "img_pedestrian.png"
pedestrian_img

In [ ]:
boto3.client("s3").download_file(jumpstart_assets_s3_bucket, f"{pedestrian_img_key_prefix}/{pedestrian_img}", pedestrian_img)

In [ ]:
def query(model_predictor, image_file_name):
    with open(image_file_name, "rb") as file:
        input_img_rb = file.read()
    query_response = model_predictor.predict(
        input_img_rb,
        {   "ContentType": "application/x-image",
            "Accept": "application/json;verbose;n_predictions=2"})
    return query_response

In [ ]:
def parse_response(query_response):
    response_dict = json.loads(query_response)
    return (
        response_dict["ids"],
        response_dict["scores"],
        response_dict["bboxes"],
        response_dict["masks"],
        response_dict["labels"],
        response_dict["image_with_masks"])

In [ ]:
query_response = query(base_model_predictor, pedestrian_img)
query_response

In [ ]:
ids, scores, bboxes, masks, labels, image_with_masks = parse_response(query_response)

In [ ]:
ids

In [ ]:
scores

In [ ]:
bboxes

In [ ]:
masks

In [ ]:
labels

In [ ]:
image_with_masks

In [ ]:
def plot_bbox(
    img,
    bboxes,
    scores=None,
    labels=None,
    thresh=0.5,
    class_names=None,
    colors=None,
    ax=None,
    linewidth=3.5,
    fontsize=12,
):
    from matplotlib import pyplot as plt
    import random
    img = img.copy()
    ax.imshow(img.astype(np.uint8))
    colors = dict()
    for i, bbox in enumerate(bboxes):
        if scores.flat[i] < thresh or labels.flat[i] < 0:
            continue
        cls_id = int(labels.flat[i]) if labels is not None else -1
        if cls_id not in colors:
            if class_names is not None:
                colors[cls_id] = plt.get_cmap("hsv")(cls_id / len(class_names))
            else:
                colors[cls_id] = (random.random(), random.random(), random.random())
        xmin, ymin, xmax, ymax = [int(x) for x in bbox]
        rect = plt.Rectangle(
            (xmin, ymin),
            xmax - xmin,
            ymax - ymin,
            fill=False,
            edgecolor=colors[cls_id],
            linewidth=linewidth,
        )
        ax.add_patch(rect)
        if class_names is not None and cls_id < len(class_names):
            class_name = class_names[cls_id]
        else:
            class_name = str(cls_id) if cls_id >= 0 else ""
        score = "{:.3f}".format(scores.flat[i]) if scores is not None else ""
        if class_name or score:
            ax.text(
                xmin,
                ymin - 2,
                "{:s} {:s}".format(class_name, score),
                bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                fontsize=fontsize,
                color="white",
            )
    return ax

In [ ]:
ids, scores, bboxes, masks, image_with_masks = (
    np.array(ids),
    np.array(scores),
    np.array(bboxes),
    np.array(masks),
    np.array(image_with_masks))

In [ ]:
ids

In [ ]:
scores

In [ ]:
bboxes

In [ ]:
masks

In [ ]:
image_with_masks

In [ ]:
width, height = image_with_masks.shape[1], image_with_masks.shape[0]

In [ ]:
width

In [ ]:
height

In [ ]:
fig = plt.figure(figsize=(20, 20))
ax = fig.add_subplot(1, 1, 1)
ax = plot_bbox(image_with_masks, bboxes, scores, ids, class_names=labels, ax=ax)
plt.show()

In [ ]:
base_model_predictor.delete_model()

In [ ]:
base_model_predictor.delete_endpoint()